In [ ]:
using BenchmarkTools
using Pandas
using PyCall
using PyPlot
using JSON

In [ ]:
obj = JSON.parse(open("../benchmark.json"));

In [ ]:
res = BenchmarkTools.load(IOBuffer(obj["benchmarkgroup"]));

In [ ]:
# https://seaborn.pydata.org/examples/grouped_boxplot.html
# + simple boxplot for a 10-state HMM / 5000 observations

In [ ]:
df = []

for K in 2:2:10
    for f in ["forward", "backward", "viterbi"]
        push!(df, ["hmmbase", f, K, minimum(res[1]["hmmbase"][f]["false"]["$K"].times)])
    end

    for f in ["forward", "backward", "viterbi"]
        push!(df, ["hmmbase (log)", f, K, minimum(res[1]["hmmbase"][f]["true"]["$K"].times)])
    end
    
    for (f, fp) in ["_do_forward_pass" => "forward", "_do_backward_pass" => "backward", "_do_viterbi_pass" => "viterbi"]
        push!(df, ["hmmlearn", fp, K, minimum(res[1]["hmmlearn"][f]["$K"].times)])
    end

    for (f, fp) in [
            "PyObject <function HMMStatesPython._messages_forwards_normalized at 0x7fd27e075560>" => "forward",
            "PyObject <function HMMStatesPython._messages_backwards_normalized at 0x7fd27e075440>" => "backward",
            "viterbi" => "viterbi"
    ]
        push!(df, ["pyhsmm", fp, K, minimum(res[1]["pyhsmm"][f]["$K"].times)])
    end
end

df = DataFrame(df, columns = ["module", "method", "K", "mintime"]);

In [ ]:
sns = pyimport("seaborn")
sns.set_style("whitegrid")

In [ ]:
figure(figsize=(7,4))
sns.barplot(x="method", y="mintime", hue="module", data=df[df.K .== 10], palette=sns.color_palette("Paired")[[2,1,7,5]])
gca().set(yscale="log")
xlabel("")
ylabel("Minimum Time")
yticks([1e6, 1e7, 1e8], ["1 ms", "10 ms", "100 ms"])
legend(frameon=false)
title("10-state HMM - 5000 observations")
savefig("benchmark_summary.png", bbox_inches = "tight", dpi = 300)